source of data: [Espérance de vie en 2024: Comparaisons régionales et départementales](https://www.insee.fr/fr/statistiques/2012749)<br>
[List of French departments by life expectancy](https://en.wikipedia.org/wiki/List_of_French_departments_by_life_expectancy) / 
[Продолжительность жизни в департаментах Франции](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_департаментах_Франции)<br />
[MapChart](https://www.mapchart.net/france-departments.html)

In [2]:
import pandas as pd
from collections import namedtuple
import json

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [3]:
pd.options.display.min_rows = 6
pd.options.display.max_rows = 50

In [4]:
CREATE_LEGEND = True
YEAR = 2024

In [5]:
CountryGroup = namedtuple('CountryGroup', ['group_label', 'color', 'countries'])

In [6]:
df = pd.read_excel(f'data/France-{YEAR}.xlsx', sheet_name='DEP', skiprows=4, skipfooter=1,
                   usecols="A:D", index_col='Unnamed: 0', na_values='nd')
df.rename(columns={'Unnamed: 1': 'department', 'Hom.':'male', 'Fem.':'female'}, inplace=True)
df

,department,male,female
01,Ain,81.4,86.0
02,Aisne,77.7,83.7
03,Allier,78.8,85.5
...,...,...,...
976,Mayotte,72.8,75.8
F,France,80.0,85.6
Champ : France.,NaN,NaN,NaN


In [7]:
df['ar_mean'] = ((df['female'] + df['male']) / 2).round(2)  # ! without rounding it is possible error during sorting
df

,department,male,female,ar_mean
01,Ain,81.4,86.0,83.70
02,Aisne,77.7,83.7,80.70
03,Allier,78.8,85.5,82.15
...,...,...,...,...
976,Mayotte,72.8,75.8,74.30
F,France,80.0,85.6,82.80
Champ : France.,NaN,NaN,NaN,NaN


In [8]:
df.drop(index=['P', '971', '972', '973', '974', '976', 'F'], inplace=True)
df

,department,male,female,ar_mean
01,Ain,81.4,86.0,83.70
02,Aisne,77.7,83.7,80.70
03,Allier,78.8,85.5,82.15
...,...,...,...,...
95,Val-d'Oise,81.1,85.3,83.20
M,France métropolitaine,80.1,85.7,82.90
Champ : France.,NaN,NaN,NaN,NaN


<br />
<br />

In [10]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
# mal.min_and_max_values(df[['male', 'female', 'ar_mean']], row_center=["13", "M"], nmb=3, max_lng=9)

In [11]:
# Some inner statistics for interest:
def min_and_max(df, region_center=''):
    print(f"Number of values: {len(df)}")
    
    def find_3_largest(df):
        t_df = df.nlargest(3)
        t_df = t_df.reset_index()
        return t_df.apply(lambda v: f"{v.iloc[1]:.1f} –Dep_{v.iloc[0]}", axis=1)

    def find_3_smallest(df):
        t_df = df.nsmallest(3).iloc[::-1]
        t_df = t_df.reset_index()
        return t_df.apply(lambda v: f"{v.iloc[1]:.1f} –Dep_{v.iloc[0]}", axis=1)
    
    if region_center:
        ser_center = df.loc[region_center].map(lambda v: f" —— {v} ——")
        final_df = pd.concat([df.apply(find_3_largest), ser_center.to_frame().T, df.apply(find_3_smallest)])
        final_df.index = ['max', 'max_2', 'max_3', 'metr. France', 'min_3', 'min_2', 'min']
    else:
        final_df = pd.concat([df.apply(find_3_largest), df.apply(find_3_smallest)])
        final_df.index = ['max', 'max_2', 'max_3', 'min_3', 'min_2', 'min']
    return final_df.style.set_properties(**{'text-align': 'left'})

min_and_max(df.loc[:, ['male', 'female', 'ar_mean']], region_center='M')

Number of values: 98


,male,female,ar_mean
max,82.7 –Dep_92,87.1 –Dep_69,84.9 –Dep_92
max_2,82.4 –Dep_2A,87.1 –Dep_92,84.7 –Dep_69
max_3,82.4 –Dep_94,86.7 –Dep_49,84.5 –Dep_94
metr. France,—— 80.1 ——,—— 85.7 ——,—— 82.9 ——
min_3,77.2 –Dep_62,83.7 –Dep_02,80.8 –Dep_58
min_2,77.1 –Dep_58,83.6 –Dep_90,80.7 –Dep_02
min,77.0 –Dep_52,83.4 –Dep_62,80.3 –Dep_62


In [12]:
# remove record about metropolitan France as a whole
df.drop(index=['M'], inplace=True)

<br />
<br />
<br />

<h2> male </h2>

In [14]:
# ls = [
#     '020f19',
#     '062f4c',
#     '08436e',
#     '0b578d',
#     '0d68a9',
#     '107ac6',
#     '128be1',
#     '2e9eee',
#     '51aef0',
#     '6ebcf3',
#     '92ccf6',
#     'b3dbf8',
#     'd4ebfb',
#     'edf7fd',
#     'f8fbfe'
# ]

# current = 83.00
# step = 0.5
# for v in ls:
#     print(f"    '{current:.1f}–{current+step-0.1:.1f}' : '{v}',")
#     current -= step

In [15]:
# ls = [
#     '020f19',
#     '042136',
#     '062f4c',
#     '073a5e',
#     '08436e',
#     '0a4e7f',
#     '0b578d',
#     '0c609c',
#     '0d68a9',
#     '0e71b7',
#     '107ac6',
#     '1182d4',
#     '128be1',
#     '1c95ec',
#     '2e9eee',
#     '40a6ef',
#     '51aef0',
#     '61b5f2',
#     '6ebcf3',
#     '80c4f4',
#     '92ccf6',
#     'a2d4f7',
#     'b3dbf8',
#     'c2e3fa',
#     'd4ebfb',
#     'e4f2fc',
#     'edf7fd',
#     'f4fafe',
#     'f8fbfe'
# ]

# current = 83.00
# step = 0.25
# for v in ls:
#     print(f"    '{current:.2f}–{current+step-0.01:.2f}' : '{v}',")
#     current -= step

In [16]:
dd_legend_male = {
    '83.0–83.4' : '020f19',
    '82.5–82.9' : '062f4c',
    '82.0–82.4' : '08436e',
    '81.5–81.9' : '0b578d',
    '81.0–81.4' : '0d68a9',
    '80.5–80.9' : '107ac6',
    '80.0–80.4' : '128be1',
    '79.5–79.9' : '2e9eee',
    '79.0–79.4' : '51aef0',
    '78.5–78.9' : '6ebcf3',
    '78.0–78.4' : '92ccf6',
    '77.5–77.9' : 'b3dbf8',
    '77.0–77.4' : 'd4ebfb',
    '76.5–76.9' : 'edf7fd',
    '76.0–76.4' : 'f8fbfe'
}


def create_legend_code(dd_legend):
    for k, v in dd_legend.items():
        print(f"{{{{Legend|#{v}|{k}}}}}")  

if CREATE_LEGEND:
    create_legend_code(dd_legend_male)

{{Legend|#020f19|83.0–83.4}}
{{Legend|#062f4c|82.5–82.9}}
{{Legend|#08436e|82.0–82.4}}
{{Legend|#0b578d|81.5–81.9}}
{{Legend|#0d68a9|81.0–81.4}}
{{Legend|#107ac6|80.5–80.9}}
{{Legend|#128be1|80.0–80.4}}
{{Legend|#2e9eee|79.5–79.9}}
{{Legend|#51aef0|79.0–79.4}}
{{Legend|#6ebcf3|78.5–78.9}}
{{Legend|#92ccf6|78.0–78.4}}
{{Legend|#b3dbf8|77.5–77.9}}
{{Legend|#d4ebfb|77.0–77.4}}
{{Legend|#edf7fd|76.5–76.9}}
{{Legend|#f8fbfe|76.0–76.4}}


<br />
<br />

In [18]:
def filter_df(df, selected_col='male'):
    filtered_df = df.loc[:, [selected_col]]   \
                    .dropna()

    filtered_df['group_label'] = filtered_df[selected_col].map(lambda x: f"{x * 4 // 2 / 2:.1f}–{x * 4 // 2 / 2 + 0.4:.1f}")
    
    # filtered_df['group_label'] = filtered_df['group_label'].replace(['76.0–76.4', '76.5–76.9'], '76.0–76.9')
    
    min_value = filtered_df[selected_col].min()
    max_value = filtered_df[selected_col].max()
    
    print(f"           ——— {selected_col} ———")
    print(f"Range: {min_value:.1f} – {max_value:.1f}   " +
          f"(Dep_{filtered_df[selected_col].idxmin()} – Dep_{filtered_df[selected_col].idxmax()})")
    print(f"Number of groups: {filtered_df['group_label'].nunique()}")
    print(f"Number of values: {len(filtered_df)}")

    return filtered_df

df_male = filter_df(df, selected_col='male')
df_male

           ——— male ———
Range: 77.0 – 82.7   (Dep_52 – Dep_92)
Number of groups: 12
Number of values: 96


,male,group_label
01,81.4,81.0–81.4
02,77.7,77.5–77.9
03,78.8,78.5–78.9
...,...,...
93,80.5,80.5–80.9
94,82.4,82.0–82.4
95,81.1,81.0–81.4


In [19]:
def extract_indexes(subdf, dd_legend=dd_legend_male):
    group_label = subdf['group_label'].iloc[0]
    countries = subdf.index.to_list()
    color = (dd_legend[group_label])
    
    ls_grouping.append(CountryGroup(group_label=group_label, countries=countries, color=color))

    return pd.Series([color, countries], index=['color', 'regions'])


ls_grouping = []
df_grouped = df_male.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_male).loc[::-1] \
                    .rename(index={'76.0–76.9': '<77'})

if ls_grouping[0].group_label == '76.0–76.9':
    ls_grouping[0] = ls_grouping[0]._replace(group_label = '<77')

df_grouped

C:\Users\L3M\AppData\Local\Temp\ipykernel_12668\2880209183.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_male.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_male).loc[::-1] \


,color,regions
group_label,,
82.5–82.9,062f4c,[92]
82.0–82.4,08436e,"[2A, 31, 69, 75, 78, 94]"
81.5–81.9,0b578d,"[38, 74, 91]"
81.0–81.4,0d68a9,"[01, 64, 73, 81, 95]"
80.5–80.9,107ac6,"[05, 06, 12, 13, 2B, 33, 35, 37, 49, 77, 83, 93]"
80.0–80.4,128be1,"[19, 21, 26, 32, 34, 40, 42, 44, 45, 53, 67, 84]"
79.5–79.9,2e9eee,"[04, 07, 15, 17, 25, 30, 46, 47, 56, 60, 63, 6..."
79.0–79.4,51aef0,"[14, 16, 41, 43, 51, 54, 71, 79, 82, 86]"
78.5–78.9,6ebcf3,"[03, 09, 11, 24, 28, 29, 39, 48, 50, 57, 66, 7..."


In [20]:
def create_map_code(ls_grouping, title='', legend_color='#000'):  # (ls_grouping, title='', legend_color='#000'):
    false, true = False, True

    jo = {
        "groups": { },
        "title": title,
        "hidden": [],
        "background": "#ffffff",
        "borders": "#000",
        "legendFont": "Century Gothic",
        "legendFontColor": legend_color,
        "legendBorderColor": "#00000000",
        "legendBgColor": "#00000000",
        "legendWidth": 160,
        "legendBoxShape": "square",
        "areBordersShown": true,
        "defaultColor": "#d1dbdd",
        "labelsColor": "#6a0707",
        "labelsFont": "Arial",
        "strokeWidth": "medium",
        "areLabelsShown": false,
        "uncoloredScriptColor": "#ffff33",
        "v6": true,
        "areStateBordersShown": true,
        "legendPosition": "custom",
        "legendX": 60,
        "legendY": 80,
        "legendSize": "larger",
        "legendTranslateX": "0.00",
        "legendStatus": "show",
        "scalingPatterns": true,
        "legendRowsSameColor": true,
        "legendColumnCount": 2
    }

    for group_label, color, regions in ls_grouping[::-1]:
        jo["groups"][f"#{color}"] = {"label": group_label,
                                     "paths": [f"Dep_{department}" for department in regions]}       

    return jo


jo = create_map_code(ls_grouping, title=f'{YEAR}: ♂', legend_color='#333333')  # legend_color='#0000bf'

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{YEAR}, departments, male.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

# print(pretty_jo)

<br />
<br />
<br />

<h2> female </h2>

In [22]:
# ls = [
#     '1e0204',
#     '60070f',
#     'a30d19',
#     'd81132',
#     'ef3b59',
#     'f2657d',
#     'f58698',
#     'f7a5b2',
#     'f9bfc6',
#     'fbd8db',
#     'fdebed',
#     'fef8f9'
# ]

# current = 88.00
# step = 0.5
# for v in ls:
#     print(f"    '{current:.1f}–{current+step-0.1:.1f}' : '{v}',")
#     current -= step

In [23]:
# ls = [
#     '1e0204',
#     '41050a',
#     '60070f',
#     '840a14',
#     'a30d19',
#     'bd0f25',
#     'd81132',
#     'ec183b',
#     'ef3b59',
#     'f1536e',
#     'f2657d',
#     'f3768b',
#     'f58698',
#     'f696a6',
#     'f7a5b2',
#     'f8b3be',
#     'f9bfc6',
#     'facdd3',
#     'fbd8db',
#     'fce2e4',
#     'fdebed',
#     'fef3f4',
#     'fef8f9'
# ]

# current = 88.00
# step = 0.25
# for v in ls:
#     print(f"    '{current:.2f}–{current+step-0.01:.2f}' : '{v}',")
#     current -= step

In [24]:
dd_legend_female = {
    '88.0–88.4' : '1e0204',
    '87.5–87.9' : '60070f',
    '87.0–87.4' : 'a30d19',
    '86.5–86.9' : 'd81132',
    '86.0–86.4' : 'ef3b59',
    '85.5–85.9' : 'f2657d',
    '85.0–85.4' : 'f58698',
    '84.5–84.9' : 'f7a5b2',
    '84.0–84.4' : 'f9bfc6',
    '83.5–83.9' : 'fbd8db',
    '83.0–83.4' : 'fdebed',
    '82.5–82.9' : 'fef8f9'
}

if CREATE_LEGEND:
    create_legend_code(dd_legend_female)

{{Legend|#1e0204|88.0–88.4}}
{{Legend|#60070f|87.5–87.9}}
{{Legend|#a30d19|87.0–87.4}}
{{Legend|#d81132|86.5–86.9}}
{{Legend|#ef3b59|86.0–86.4}}
{{Legend|#f2657d|85.5–85.9}}
{{Legend|#f58698|85.0–85.4}}
{{Legend|#f7a5b2|84.5–84.9}}
{{Legend|#f9bfc6|84.0–84.4}}
{{Legend|#fbd8db|83.5–83.9}}
{{Legend|#fdebed|83.0–83.4}}
{{Legend|#fef8f9|82.5–82.9}}


In [25]:
df_female = filter_df(df, selected_col='female')
df_female

           ——— female ———
Range: 83.4 – 87.1   (Dep_62 – Dep_69)
Number of groups: 9
Number of values: 96


,female,group_label
01,86.0,86.0–86.4
02,83.7,83.5–83.9
03,85.5,85.5–85.9
...,...,...
93,85.6,85.5–85.9
94,86.7,86.5–86.9
95,85.3,85.0–85.4


In [26]:
ls_grouping = []
df_grouped = df_female.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_female).loc[::-1]

df_grouped

C:\Users\L3M\AppData\Local\Temp\ipykernel_12668\2493027205.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_female.groupby(['group_label']).apply(extract_indexes, dd_legend=dd_legend_female).loc[::-1]


,color,regions
group_label,,
87.0–87.4,a30d19,"[69, 92]"
86.5–86.9,d81132,"[31, 37, 49, 73, 74, 75, 78, 94]"
86.0–86.4,ef3b59,"[01, 04, 05, 06, 2A, 34, 35, 38, 53, 64, 81, 8..."
85.5–85.9,f2657d,"[03, 13, 17, 2B, 21, 26, 33, 40, 42, 44, 46, 4..."
85.0–85.4,f58698,"[07, 12, 30, 32, 41, 43, 45, 47, 50, 56, 61, 6..."
84.5–84.9,f7a5b2,"[09, 10, 11, 14, 15, 16, 19, 24, 25, 27, 28, 2..."
84.0–84.4,f9bfc6,"[22, 23, 57, 58, 59, 76, 88]"
83.5–83.9,fbd8db,"[02, 08, 18, 80, 90]"
83.0–83.4,fdebed,[62]


In [27]:
jo = create_map_code(ls_grouping, title=f'{YEAR}: ♀', legend_color='#333333')  # legend_color='#c20000'

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{YEAR}, departments, female.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

# print(pretty_jo)

♂ <br />
♀